In [1]:
# initial imports:
import sys, os

# tensiometer local imports:
here = os.path.dirname(os.getcwd())
tensiometer_path = os.path.join(here, 'tensiometer')
sys.path.append(tensiometer_path)

import tensiometer
print('Tensiometer version:', tensiometer.__version__)
print('Tensiometer path:', tensiometer.__path__)

Tensiometer version: 1.0.2
Tensiometer path: ['/Users/marco/Desktop/IW/ParametricModels/tensiometer/tensiometer']


In [2]:
# chains paths:

chains_dir = os.path.join('chains')

In [3]:
# getdist settings:

getdist_settings = {
    'smooth_scale_1D': 0.3,
    'boundary_correction_order': 1,
    'mult_bias_correction_order': 1,
    'smooth_scale_2D': 0.3,
    }

In [5]:
#import cobaya
from cobaya import load_samples
from cobaya.model import get_model

name = '01p1_LCDM_BAO_DESIDR2'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))

01p1_LCDM_BAO_DESIDR2: R = 0.0081
